In [1]:
import torch
from transformers import MarianMTModel, MarianTokenizer, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
import evaluate

C:\Users\jerno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = "Iker/Document-Translation-en-es"

summary_model_name = "t5-small"
translation_model_name = "Helsinki-NLP/opus-mt-en-es"
revision = "main"

tokenizer_model_name = "distilbert-base-uncased"
model_name = "Helsinki-NLP/opus-mt-en-es"

device = 0 if torch.cuda.is_available() else -1

## Base summariser

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained(summary_model_name)
tokenizer = AutoTokenizer.from_pretrained(summary_model_name)
text = """
The Hugging Face Transformers library provides state-of-the-art general-purpose architectures
for natural language understanding (NLU) and natural language generation (NLG). These architectures
include BERT, GPT, GPT-2, BART, and T5, which can be applied to text classification, summarization, translation, and more.
"""

inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

the Hugging Face Transformers library provides state-of-the-art architectures for natural language understanding (NLU) and natural language generation (NLG). these architectures include BERT, GPT, GPT-2, and BART.


## Baseline translator

In [4]:
tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
model = MarianMTModel.from_pretrained(translation_model_name)
english_text = "Let's learn AI using Python"
input_ids = tokenizer.encode(english_text, return_tensors="pt")
translated_ids = model.generate(input_ids)

translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

print("Translated Text (Spanish):", translated_text)

Translated Text (Spanish): Vamos a aprender IA usando Python


## Load dataset

In [5]:
dataset = load_dataset(dataset_name)
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"].shuffle(seed=42).select(range(100))
test_dataset = split_dataset["test"].shuffle(seed=42).select(range(100))

## x/y split

In [6]:
train_x_data = train_dataset["en"]
train_y_data = train_dataset["es"]
test_x_data = test_dataset["en"]
test_y_data = test_dataset["es"]

# Load Pre-Trained Models

In [7]:
model = MarianMTModel.from_pretrained(model_name)
bleu = evaluate.load("sacrebleu")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)

## Load tokenizer

In [8]:
# I used the original text (english) to tokenize the data, seemed logical to me, but idk
def tokenize_function(examples):
    return tokenizer(examples["en"], padding="max_length", truncation=True)

train_x_data = train_dataset.map(tokenize_function, batched=True)
test_x_data = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 100/100 [00:00<00:00, 458.86 examples/s]


# Evaluate Model

In [9]:
# Okay this maybe works, maybe not, who knows, help me?
input_ids = torch.tensor(test_x_data["input_ids"]).to(model.device)
print('translating the tokens..')
translated_tokens = model.generate(input_ids=input_ids)
predicted_translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

references = [[ref] for ref in test_y_data] #The bleu score needs a list of lists
bleu_score = bleu.compute(predictions=predicted_translations, references=references)

print(f"BLEU score: {bleu_score['score']}")

getting input ids..
translating the tokens..


KeyboardInterrupt: 